## User Experience Analytics

### Import Liberaries

In [1]:
import pickle
import numpy as np
import pandas as pd
from math import floor
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import zscore
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import sys, os

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from df_outlier import DfOutlier
from vis_seaborn import *

from vis_plotly import *

### Loading Data

In [ ]:
df1 = pd.read_csv("../data/my_clean_data.csv")
df1.info()

1. 

In [ ]:
user_experience = df1[[
    "MSISDN/Number",
    "Avg RTT DL (ms)",
    "Avg RTT UL (ms)",
    "Avg Bearer TP DL (kbps)",
    "Avg Bearer TP UL (kbps)",
    "TCP DL Retrans. Vol (Bytes)",
    "TCP UL Retrans. Vol (Bytes)",
    "Handset Type"]].copy()

In [ ]:
user_experience['total_avg_rtt'] = user_experience['Avg RTT DL (ms)'] + user_experience['Avg RTT UL (ms)']
user_experience['total_avg_tp'] = user_experience['Avg Bearer TP DL (kbps)'] + user_experience['Avg Bearer TP UL (kbps)']
user_experience['total_avg_tcp'] = user_experience['TCP DL Retrans. Vol (Bytes)'] + user_experience['TCP UL Retrans. Vol (Bytes)']
user_experience.info()

In [ ]:
_user_experience = user_experience.groupby('MSISDN/Number').agg({
    'total_avg_rtt': 'sum',
    'total_avg_tp': 'sum',
    'total_avg_tcp': 'sum',
    'Handset Type': [lambda x: x.mode()[0]]})

user_experience = pd.DataFrame(columns=[
    "total_avg_rtt",
    "total_avg_tp",
    "total_avg_tcp",
    "Handset Type"])

user_experience["total_avg_rtt"] = _user_experience["total_avg_rtt"]['sum']
user_experience["total_avg_tp"] = _user_experience["total_avg_tp"]['sum']
user_experience["total_avg_tcp"] = _user_experience["total_avg_tcp"]['sum']
user_experience["Handset Type"] = _user_experience["Handset Type"]['<lambda>']
user_experience.head()

In [ ]:
2. 

In [ ]:
sorted_by_tcp = user_experience.sort_values(
    'total_avg_tcp', ascending=False)
top_10 = sorted_by_tcp.head(10)['total_avg_tcp']
last_10 = sorted_by_tcp.tail(10)['total_avg_tcp']
most_10 = user_experience['total_avg_tcp'].value_counts().head(10)


In [ ]:
mult_hist([top_10, last_10, most_10], 1,
          3, "TCP values in the dataset", ['Top 10', 'Last 10', 'Most 10'])

In [ ]:
sorted_by_rtt = user_experience.sort_values(
    'total_avg_rtt', ascending=False)
top_10 = sorted_by_rtt.head(10)['total_avg_rtt']
last_10 = sorted_by_rtt.tail(10)['total_avg_rtt']
most_10 = user_experience['total_avg_rtt'].value_counts().head(10)

In [ ]:

mult_hist([top_10, last_10, most_10, top_10], 1,
          3, "TCP values in the dataset", ['Top 10', 'Last 10', 'Most 10'])

In [ ]:
sorted_by_tp = user_experience.sort_values(
    'total_avg_tp', ascending=False)
top_10 = sorted_by_tp.head(10)['total_avg_tp']
last_10 = sorted_by_tp.tail(10)['total_avg_tp']
most_10 = user_experience['total_avg_tp'].value_counts().head(10)

In [ ]:
ult_hist([top_10, last_10, most_10, top_10], 1,
          3, "TCP values in the dataset", ['Top 10', 'Last 10', 'Most 10'])

In [ ]:
3. 

In [ ]:
handset_type_df = user_experience.groupby('Handset Type').agg(
    {'total_avg_tp': 'mean', 'total_avg_tcp': 'mean'})
handset_type_df.head()

In [ ]:
sorted_by_tp = handset_type_df.sort_values(
    'total_avg_tp', ascending=False)
top_tp = sorted_by_tp['total_avg_tp']

In [ ]:
hist(top_tp, interactive=True)


In [ ]:
hist(top_tp.head(20))

In [ ]:
sorted_by_tcp = handset_type_df.sort_values(
    'total_avg_tcp', ascending=False)
top_tcp = sorted_by_tcp['total_avg_tcp']